# Convolutional Neural Network 실습 - MXNet ``gluon``

## 손으로 쓴 숫자 인식 (Handwritten Digit Recognition)


이번 실습은 ``gluon``을 이용해서 convolutional neural network를 구성하는 방법에 대해서 살펴보겠습니다.

In [ ]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

## 실행 환경 설정 (GPU)

G2, P2 인스턴스를 이용하는 경우 mx.gpu()를 이용해서 GPU 연산이 될 수 있도록하고, C4와 같은 GPU가 없는 인스턴스를 이용하는 경우 mx.cpu()로 CPU를 이용하도록 합니다.

In [ ]:
ctx = mx.gpu()

## MNIST dataset 준비하기

MXNet Gluon 패키지를 이용해서 MNIST dataset을 로딩하고, 학습 dataset과 테스트 dataset을 준비합니다.

In [ ]:
batch_size = 64
#num_inputs = 784
num_outputs = 10
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

## Convolutional Neural Network 정의

``gluon.nn``을 이용해서 Convolutional layer, Pooling layer 그리고 Fully connected layer를 추가합니다.

> <font color='red'>문제</font>
> Convolutional layer에 적용된 Activation 함수를 sigmoid에서 relu로 변경하세요.

In [ ]:
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='sigmoid'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='sigmoid'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

## 파라메터 초기화

Xavier 초기화 방법을 이용해서 파라메터들을 초기화합니다. 다른 초기화들은 [MXNet - Optimization](https://mxnet.incubator.apache.org/api/python/optimization.html)에서 확인해보세요.

In [ ]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

## Softmax cross-entropy Loss

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

최적화 알고리즘으로 SGD를 적용하고, learning rate를 0.1로 설정합니다.

> <font color='red'>문제</font>
> learning_rate를 0.1로 설정해서 학습을 수행한 후, learning_rate 값을 큰 값으로 설정하면 accuracy가 어떻게 되는지 살펴보세요.

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

## 정확도를 계산하기 위한 evaluation loop 정의

In [ ]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

## 학습 수행

In [ ]:
epochs = 10
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])          
        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0)) 
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
        
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))    

## 결론

``gluon``을 이용한 경우 CPU 및 GPU 모두의 경우 속도가 빠른것을 관찰할 수 있습니다. 이는, ``gluon``이 C++ 코드로 작성된 최적화된 layer를 사용하기 때문입니다.